The data imported in this file is our raw data. The files needed to run it can be found here: https://drive.google.com/drive/folders/1MEz1kqZ3AQVY_bdPxITEpsYqLlYEUBU-?usp=sharing

In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
## Import US Data
us = pd.read_csv("Youtube Data/USVideos.csv")
us_categories = pd.read_json("Youtube Data/US_category_id.json")
## Import CA Data
ca = pd.read_csv("Youtube Data/CAVideos.csv")
ca_categories = pd.read_json("Youtube Data/CA_category_id.json")
## Import GB Data
gb = pd.read_csv("Youtube Data/GBVideos.csv")
gb_categories = pd.read_json("Youtube Data/GB_category_id.json")
## Import FR Data
fr = pd.read_csv("Youtube Data/FRVideos.csv")
fr_categories = pd.read_json("Youtube Data/FR_category_id.json")
## Import DE Data
de = pd.read_csv("Youtube Data/DEVideos.csv")
de_categories = pd.read_json("Youtube Data/DE_category_id.json")

In [3]:
## Get Music Data for Each Country
us_music = us[us.category_id == 10]
ca_music = ca[ca.category_id == 10]
gb_music = gb[gb.category_id == 10]
fr_music = fr[fr.category_id == 10]
de_music = de[de.category_id == 10]

In [4]:
## list of all dates recorded
dates = us.trending_date.unique()
## create dictionary to store lists of countries that a patricular video was trending in on a particular day
country_dict = {}
for date in dates:
    country_dict[date] = {}

def append_countries(data, country):
    for index, row in data.iterrows(): 
        if row.video_id not in country_dict[row.trending_date]:
            country_dict[row.trending_date][row.video_id] = [country]
        else:
            country_dict[row.trending_date][row.video_id].append(country)
append_countries(us_music, "US")
append_countries(ca_music, "CA")
append_countries(gb_music, "GB")
append_countries(fr_music, "FR")
append_countries(de_music, "DE")

In [5]:
## combine all of the countries
combined = us_music.append(ca_music).append(gb_music).append(fr_music).append(de_music)

## delete any duplicate rows from the dataframe
combined = combined.drop_duplicates()

## set new sequential indices
idx = [*range(0, len(combined))]
combined["index"] = idx
combined = combined.set_index("index")

## delete the category id because we are only using one category
## and delete thumbnail like because we don't need it
combined = combined.drop(["category_id", "thumbnail_link", "description"], axis=1)

In [6]:
combined.head()

video_id trending_date  \
index                              
0      5E4ZBSInqUU      17.14.11   
1      n1WpP7iowLc      17.14.11   
2      e_7zHm7GsYc      17.14.11   
3      zZ9FciUx6gs      17.14.11   
4      PaJCFHXcWmM      17.14.11   

                                                   title channel_title  \
index                                                                    
0             Marshmello - Blocks (Official Music Video)    marshmello   
1             Eminem - Walk On Water (Audio) ft. Beyoncé    EminemVEVO   
2      Hunter Hayes - You Should Be Loved (Part One O...  Hunter Hayes   
3      Nickelback - The Betrayal Act III [Official Vi...    Nickelback   
4                                      U2 - The Blackout        U2VEVO   

                   publish_time  \
index                             
0      2017-11-13T17:00:00.000Z   
1      2017-11-10T17:00:03.000Z   
2      2017-11-13T15:01:18.000Z   
3      2017-11-13T15:31:44.000Z   
4      2017-11-13T17:00:04.000Z   

                                                    tags     views   likes  \
index                                                                        
0      marshmello|"blocks"|"marshmello blocks"|"block...    687582  114188   
1      Eminem|"Walk"|"On"|"Water"|"Aftermath/Shady/In...  17158531  787419   
2      Hunter|"Hayes"|"you should be loved"|"the shad...     13917    1318   
3      Nickelback|"Feed The Machine"|"The Betrayal Ac...     57169    6927   
4          U2|"The"|"Blackout"|"Island"|"Records"|"Rock"     60506    5389   

       dislikes  comment_count  comments_disabled  ratings_disabled  \
index                                                                 
0          1333           8371              False             False   
1         43420         125882              False             False   
2            24             76              False             False   
3           161            565              False             False   
4           106            455              False             False   

       video_error_or_removed  
index                          
0                       False  
1                       False  
2                       False  
3                       False  
4                       False

In [7]:
## add countries as a column 
combined["countries_trending_in"] = ""
for i, row in combined.iterrows():
    combined.at[i, "countries_trending_in"] = country_dict[row.trending_date][row.video_id]

In [8]:
combined.head(20)

video_id trending_date  \
index                              
0      5E4ZBSInqUU      17.14.11   
1      n1WpP7iowLc      17.14.11   
2      e_7zHm7GsYc      17.14.11   
3      zZ9FciUx6gs      17.14.11   
4      PaJCFHXcWmM      17.14.11   
5      0tO_l_Ed5Rs      17.14.11   
6      9t9u_yPEidY      17.14.11   
7      ujyTQNNjjDU      17.14.11   
8      2Vv-BfVoq4g      17.14.11   
9      lY_0mkYDZDU      17.14.11   
10     ObIQ0s02UHg      17.14.11   
11     _Iz83-Cmt6A      17.14.11   
12     YlvCVbfS9M0      17.14.11   
13     e4FApt6z55c      17.14.11   
14     2XK4omx9uMU      17.14.11   
15     RkHuWjiR-LM      17.14.11   
16     DIU3xPdhCBI      17.14.11   
17     pz95u3UVpaM      17.14.11   
18     viyRD5z6ilQ      17.14.11   
19     QFfEtKvXMAs      17.14.11   

                                                   title        channel_title  \
index                                                                           
0             Marshmello - Blocks (Official Music Video)           marshmello   
1             Eminem - Walk On Water (Audio) ft. Beyoncé           EminemVEVO   
2      Hunter Hayes - You Should Be Loved (Part One O...         Hunter Hayes   
3      Nickelback - The Betrayal Act III [Official Vi...           Nickelback   
4                                      U2 - The Blackout               U2VEVO   
5      Matthew Santoro - FACTS (Official Music Video)...       MatthewSantoro   
6      Jennifer Lopez - Amor, Amor, Amor (Official Vi...    JenniferLopezVEVO   
7                     G-Eazy - The Plan (Official Video)       GEazyMusicVEVO   
8            Ed Sheeran - Perfect (Official Music Video)           Ed Sheeran   
9      Foster The People - Sit Next to Me (Official V...  fosterthepeopleVEVO   
10     Jason Derulo - Tip Toe feat. French Montana (O...         Jason Derulo   
11     Little Big Town with Jimmy Webb - Wichita Line...              CMAVEVO   
12      Alan Walker - All Falls Down (Behind The Scenes)          Alan Walker   
13      Kimbra - Top of the World (Official Music Video)          kimbramusic   
14     Camila Cabello COMPLETELY NAILS 'Finish The Ly...           Capital FM   
15          Neck Deep - Parachute (Official Music Video)     Hopeless Records   
16     Sam Smith surprises brides at their wedding! (...            BBC Music   
17     Camila Cabello - Havana (Vertical Video) ft. Y...    CamilaCabelloVEVO   
18                     Luke Bryan - O Holy Night (Audio)        LukeBryanVEVO   
19              Niall Horan - Too Much To Ask (Acoustic)       NiallHoranVEVO   

                   publish_time  \
index                             
0      2017-11-13T17:00:00.000Z   
1      2017-11-10T17:00:03.000Z   
2      2017-11-13T15:01:18.000Z   
3      2017-11-13T15:31:44.000Z   
4      2017-11-13T17:00:04.000Z   
5      2017-11-11T16:00:01.000Z   
6      2017-11-10T15:00:00.000Z   
7      2017-11-10T05:00:01.000Z   
8      2017-11-09T11:04:14.000Z   
9      2017-11-10T17:00:05.000Z   
10     2017-11-10T14:40:48.000Z   
11     2017-11-09T22:23:51.000Z   
12     2017-11-11T15:11:18.000Z   
13     2017-11-10T15:43:00.000Z   
14     2017-11-10T14:40:32.000Z   
15     2017-11-10T18:03:54.000Z   
16     2017-11-09T22:26:10.000Z   
17     2017-11-10T05:01:00.000Z   
18     2017-11-10T05:00:01.000Z   
19     2017-11-10T17:00:05.000Z   

                                                    tags     views    likes  \
index                                                                         
0      marshmello|"blocks"|"marshmello blocks"|"block...    687582   114188   
1      Eminem|"Walk"|"On"|"Water"|"Aftermath/Shady/In...  17158531   787419   
2      Hunter|"Hayes"|"you should be loved"|"the shad...     13917     1318   
3      Nickelback|"Feed The Machine"|"The Betrayal Ac...     57169     6927   
4          U2|"The"|"Blackout"|"Island"|"Records"|"Rock"     60506     5389   
5      matthew santoro facts|"music video"|"matthew s...    328330    15186   
6      Jennifer Lopez ft. Wisin

In [9]:
tags = combined['tags']

tagsDf = pd.DataFrame(tags)


tagsDf = tagsDf.tags.replace('[^a-zA-Z0-9|"/\ ]', '%', regex=True)


filtered_tags = pd.DataFrame(tagsDf)

filtered_tags = filtered_tags[~filtered_tags.tags.str.contains("%")]






filtered_tags['tags2'] = filtered_tags.tags.str.split('|')
filtered_tags = filtered_tags.drop('tags', 1)



combined['tags'] = combined.tags.replace('[^a-zA-Z0-9|"/\ ]', '%', regex=True)

combined['tags'] = combined['tags'][~combined.tags.str.contains("%")]

filtered = combined[combined['tags'].notna()]


filtered['tags'] = filtered.tags.replace('["]', '', regex = True)

filtered['tags'] = filtered.tags.str.split('|')


filtered['publish-day'] = filtered['publish_time'].str[:10]
filtered['publish-time'] = filtered['publish_time'].str[11:19]


del filtered['publish_time']






filtered['trending_date'] = filtered.trending_date.str.split('.')




for date in filtered['trending_date']:
    date[0], date[-1] = date[-1], date[0]


final_dates = []

for date in filtered['trending_date']:
    stringDate = ''.join(date)
    stringDateFormatted = stringDate[:2] + '-' + stringDate[2:4] + '-20' + stringDate[4:]
    stringDateFormatted = stringDateFormatted.lstrip('0')
    final_dates.append(stringDateFormatted)

filtered['trending_date'] = final_dates 
print(filtered['trending_date'])


print(filtered.head())

index
0        11-14-2017
1        11-14-2017
2        11-14-2017
3        11-14-2017
4        11-14-2017
            ...    
26623     6-11-2018
26625     6-11-2018
26629     6-12-2018
26631     6-13-2018
26633     6-13-2018
Name: trending_date, Length: 12311, dtype: object
          video_id trending_date  \
index                              
0      5E4ZBSInqUU    11-14-2017   
1      n1WpP7iowLc    11-14-2017   
2      e_7zHm7GsYc    11-14-2017   
3      zZ9FciUx6gs    11-14-2017   
4      PaJCFHXcWmM    11-14-2017   

                                                   title channel_title  \
index                                                                    
0             Marshmello - Blocks (Official Music Video)    marshmello   
1             Eminem - Walk On Water (Audio) ft. Beyoncé    EminemVEVO   
2      Hunter Hayes - You Should Be Loved (Part One O...  Hunter Hayes   
3      Nickelback - The Betrayal Act III [Official Vi...    Nickelback   
4                          

/Users/veronicawertz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/veronicawertz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/veronicawertz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [10]:
filtered.to_csv(r'youtube_music.csv', index = False)